This notebook contains the code to apply `bayes-drt` (CHB algorithm) to the example spectra presented in the manuscript. The code to apply the Gaussian Process, Elastic Net, gEISi, and previous hierarchical Bayesian algorithms to the same data can be found in the `comparisons` directory.

In [1]:
import numpy as np
import pandas as pd
import os 
import sys
import glob
import time
import matplotlib.pyplot as plt

if '../bayes-drt' not in sys.path:
    sys.path.append('../bayes-drt')
import drt
from stan_models import save_pickle,load_pickle
import eis_utils as gt

%load_ext autoreload
%autoreload 2

# Simulated data

# DRT

In [267]:
dr = drt.DRT()
files = glob.glob('../data/simulated/Z_*.csv')
# trunc and RC-ZARC circuits need different basis tau
files = [f for f in files if f.find('trunc')==-1 and f.find('RC-ZARC')==-1 and f.find('DDT')==-1]
print(len(files))
# fit one file to generate A matrices
df = pd.read_csv(files[0])
Z = df['Zreal'] + 1j*df['Zimag']
dr.ridge_fit(df['Freq'].values,Z.values)

60


../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 20 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


In [96]:
"MAP fits"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    circ = file.split('_')[1]
    Zoutfile = os.path.join('map_results',f'Zout{suffix}')
    Goutfile = os.path.join('map_results',f'Gout{suffix}')
    pkl = os.path.join('map_results','obj{}.pkl'.format(suffix[:-4]))
    if file.split('_')[2]=='noiseless.csv':
        sigma_min=0.005
    else:
        sigma_min=0.002
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    df = pd.read_csv(file)
    Z = df['Zreal'].values + 1j*df['Zimag'].values
    # fit
    file_start = time.time()
    if circ=='ZARC-RL':
        nonneg_drt = False
    else:
        nonneg_drt = True
    dr.map_fit(df['Freq'].values,Z,init_from_ridge=False,sigma_min=sigma_min,nonneg_drt=nonneg_drt)
    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

    # save results
    Z_pred = dr.predict_Z(df['Freq'].values)
    Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
    sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
    Z_res['sigma_re'] = sigma_re
    Z_res['sigma_im'] = sigma_im

    g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot)
    g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                         columns=['tau','gamma'])
    Z_res.to_csv(Zoutfile,index=False)
    g_res.to_csv(Goutfile,index=False)

    # pickle the drt object for access to all other attributes
    save_pickle(dr,pkl)
        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_2RC_Macdonald_0.25.csv
-------------------------------
File fit time: 1.36 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to map_results\obj_2RC_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_2RC_Macdonald_1.0.csv
-------------------------------
File fit time: 1.24 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to map_results\obj_2RC_Macdonald_1.0.pkl
-------------------------------
../data/simulated\Z_2RC_Macdonald_2.5.csv
-------------------------------
File fit time: 1.69 seconds


...

../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)



Dumped pickle to map_results\obj_ZARC_uniform_0.25.pkl
-------------------------------
../data/simulated\Z_ZARC_uniform_1.0.csv
-------------------------------
File fit time: 2.87 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to map_results\obj_ZARC_uniform_1.0.pkl
-------------------------------
../data/simulated\Z_ZARC_uniform_2.5.csv
-------------------------------
File fit time: 1.95 seconds
Dumped pickle to map_results\obj_ZARC_uniform_2.5.pkl
Total fit time: 2.35 minutes


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [244]:
"MCMC sampling"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    circ = file.split('_')[1]
    Zoutfile = os.path.join('bayes_results',f'Zout{suffix}')
    Goutfile = os.path.join('bayes_results',f'Gout{suffix}')
    pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix[:-4]))
    if file.split('_')[2]=='noiseless.csv':
        sigma_min=0.005
    else:
        sigma_min=0.002
        
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    df = pd.read_csv(file)
    Z = df['Zreal'].values + 1j*df['Zimag'].values

    # fit
    file_start = time.time()
    if circ=='ZARC-RL':
        nonneg_drt = False
    else:
        nonneg_drt = True
    dr.bayes_fit(df['Freq'].values,Z,init_from_ridge=False,sigma_min=sigma_min,nonneg_drt=nonneg_drt)
    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

    Z_pred = dr.predict_Z(df['Freq'].values)
    Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,
                         columns=['freq','Zreal','Zimag'])
    sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
    Z_res['sigma_re'] = sigma_re
    Z_res['sigma_im'] = sigma_im

    g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot)
    # drt credibility interval
    g_lo = dr.predict_distribution('DRT',eval_tau=tau_plot,percentile=2.5)
    g_hi = dr.predict_distribution('DRT',eval_tau=tau_plot,percentile=97.5)
    g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                         columns=['tau','gamma','gamma_lo','gamma_hi'])

    Z_res.to_csv(Zoutfile,index=False)
    g_res.to_csv(Goutfile,index=False)

    # pickle the drt object for access to all other attributes
    save_pickle(dr,pkl)
        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_2RC_Macdonald_0.25.csv
-------------------------------


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 109.10 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_2RC_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_2RC_Macdonald_1.0.csv
-------------------------------


...

../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_ZARC_uniform_1.0.pkl
-------------------------------
../data/simulated\Z_ZARC_uniform_2.5.csv
-------------------------------


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 47.58 seconds
Dumped pickle to bayes_results\obj_ZARC_uniform_2.5.pkl
Total fit time: 80.34 minutes


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [271]:
"Fit example file with 4 chains, 1000 iterations for comparison"
file = '../data/simulated/Z_2RC_uniform_0.25.csv'
df = pd.read_csv(file)
Z = df['Zreal'].values + 1j*df['Zimag'].values
suffix = file[file.find('_'):-4]
Zoutfile = os.path.join('bayes_results',f'Zout{suffix}_4x1000.csv')
Goutfile = os.path.join('bayes_results',f'Gout{suffix}_4x1000.csv')
pkl = os.path.join('bayes_results',f'obj{suffix}_4x1000.pkl')
tau_plot = np.logspace(-7,2,200)

file_start = time.time()
dr.bayes_fit(df['Freq'].values,Z,warmup=500,sample=500,chains=4,nonneg_drt=True)
print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
Z_pred = dr.predict_Z(df['Freq'].values)
Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,
                     columns=['freq','Zreal','Zimag'])
sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
Z_res['sigma_re'] = sigma_re
Z_res['sigma_im'] = sigma_im

g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot)
# drt credibility interval
g_lo = dr.predict_distribution('DRT',eval_tau=tau_plot,percentile=2.5)
g_hi = dr.predict_distribution('DRT',eval_tau=tau_plot,percentile=97.5)
g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                     columns=['tau','gamma','gamma_lo','gamma_hi'])

Z_res.to_csv(Zoutfile,index=False)
g_res.to_csv(Goutfile,index=False)

# pickle the drt object for access to all other attributes
save_pickle(dr,pkl)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 515.49 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_2RC_uniform_0.25_4x1000.pkl


## RC-ZARC

In [97]:
basis_tau = np.logspace(-2,3,51)
dr_rcz = drt.DRT(basis_freq=1/(2*np.pi*basis_tau))
rcz_files = glob.glob('../data/simulated/Z_RC-ZARC*.csv')

In [98]:
"MAP fits"
# tau for plotting
tau_plot = np.logspace(np.log10(np.exp(-5)),np.log10(np.exp(5.5)),200)
start = time.time()

for file in rcz_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    Zoutfile = os.path.join('map_results',f'Zout{suffix}')
    Goutfile = os.path.join('map_results',f'Gout{suffix}')
    pkl = os.path.join('map_results','obj{}.pkl'.format(suffix[:-4]))
    if file.split('_')[2]=='noiseless.csv':
        sigma_min=0.005
    else:
        sigma_min=0.002
    if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
        print('Already ran')
    else:
        df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        # fit
        file_start = time.time()
        dr_rcz.map_fit(df['Freq'].values,Z,init_from_ridge=False,sigma_min=sigma_min,nonneg_drt=True)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

        # save results
        Z_pred = dr_rcz.predict_Z(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
        sigma_re,sigma_im = dr_rcz.predict_sigma(df['Freq'].values)
        Z_res['sigma_re'] = sigma_re
        Z_res['sigma_im'] = sigma_im

        g_pred = dr_rcz.predict_distribution('DRT',eval_tau=tau_plot)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr_rcz,pkl)
        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_RC-ZARC_Macdonald_0.25.csv
-------------------------------
File fit time: 0.96 seconds
Dumped pickle to map_results\obj_RC-ZARC_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_RC-ZARC_Macdonald_1.0.csv
-------------------------------
File fit time: 0.95 seconds
Dumped pickle to map_results\obj_RC-ZARC_Macdonald_1.0.pkl
-------------------------------
../data/simulated\Z_RC-ZARC_Macdonald_2.5.csv
-------------------------------
File fit time: 0.56 seconds
Dumped pickle to map_results\obj_RC-ZARC_Macdonald_2.5.pkl
-------------------------------
../data/simulated\Z_RC-ZARC_noiseless.csv
-------------------------------
File fit time: 0.84 seconds
Dumped pickle to map_results\obj_RC-ZARC_noiseless.pkl
-------------------------------
../data/simulated\Z_RC-ZARC_Orazem_0.25.csv
-------------------------------
File fit time: 0.72 seconds
Dumped pickle to map_results\obj_RC-ZARC_Orazem_0.25.pkl
-------------------------

In [245]:
"MCMC sampling"
# tau for plotting
tau_plot = np.logspace(np.log10(np.exp(-5)),np.log10(np.exp(5.5)),200)
start = time.time()

for file in rcz_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    Zoutfile = os.path.join('bayes_results',f'Zout{suffix}')
    Goutfile = os.path.join('bayes_results',f'Gout{suffix}')
    pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix[:-4]))
#     if file.split('_')[2]=='noiseless.csv':
#         sigma_min=0.005
#     else:
    sigma_min=0.002
        
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    df = pd.read_csv(file)
    Z = df['Zreal'].values + 1j*df['Zimag'].values

    # fit
    file_start = time.time()
    dr_rcz.bayes_fit(df['Freq'].values,Z,init_from_ridge=False,sigma_min=sigma_min,nonneg_drt=True)
    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

    Z_pred = dr_rcz.predict_Z(df['Freq'].values)
    Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,
                         columns=['freq','Zreal','Zimag'])
    sigma_re,sigma_im = dr_rcz.predict_sigma(df['Freq'].values)
    Z_res['sigma_re'] = sigma_re
    Z_res['sigma_im'] = sigma_im

    g_pred = dr_rcz.predict_distribution('DRT',eval_tau=tau_plot)
    # drt credibility interval
    g_lo = dr_rcz.predict_distribution('DRT',eval_tau=tau_plot,percentile=2.5)
    g_hi = dr_rcz.predict_distribution('DRT',eval_tau=tau_plot,percentile=97.5)
    g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                         columns=['tau','gamma','gamma_lo','gamma_hi'])

    Z_res.to_csv(Zoutfile,index=False)
    g_res.to_csv(Goutfile,index=False)

    # pickle the drt object for access to all other attributes
    save_pickle(dr_rcz,pkl)

print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_RC-ZARC_Macdonald_0.25.csv
-------------------------------


File fit time: 49.35 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_RC-ZARC_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_RC-ZARC_Macdonald_1.0.csv
-------------------------------
File fit time: 31.07 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


...

Dumped pickle to bayes_results\obj_RC-ZARC_uniform_0.25.pkl
-------------------------------
../data/simulated\Z_RC-ZARC_uniform_1.0.csv
-------------------------------
File fit time: 31.36 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_RC-ZARC_uniform_1.0.pkl
-------------------------------
../data/simulated\Z_RC-ZARC_uniform_2.5.csv
-------------------------------


File fit time: 21.42 seconds
Dumped pickle to bayes_results\obj_RC-ZARC_uniform_2.5.pkl
Total fit time: 5.94 minutes


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


## Truncated RL-ZARC-ZARC

In [268]:
dr_trunc = drt.DRT(basis_freq=np.logspace(6,-2,81))
trunc_files = glob.glob('../data/simulated/Z_trunc*.csv')

# fit one file to generate A matrices
df = pd.read_csv(trunc_files[0])
Z = df['Zreal'] + 1j*df['Zimag']
dr_trunc.ridge_fit(df['Freq'].values,Z.values)

../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 20 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


In [269]:
"MAP fits"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in trunc_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    Zoutfile = os.path.join('map_results',f'Zout{suffix}')
    Goutfile = os.path.join('map_results',f'Gout{suffix}')
    pkl = os.path.join('map_results','obj{}.pkl'.format(suffix[:-4]))
    if file.split('_')[2]=='noiseless.csv':
        sigma_min=0.005
    else:
        sigma_min=0.002
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    df = pd.read_csv(file)
    Z = df['Zreal'].values + 1j*df['Zimag'].values
    # fit
    file_start = time.time()
    dr_trunc.map_fit(df['Freq'].values,Z,sigma_min=sigma_min,nonneg_drt=False)
    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

    # save results
    Z_pred = dr_trunc.predict_Z(df['Freq'].values)
    Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
    sigma_re,sigma_im = dr_trunc.predict_sigma(df['Freq'].values)
    Z_res['sigma_re'] = sigma_re
    Z_res['sigma_im'] = sigma_im

    g_pred = dr_trunc.predict_distribution('DRT',eval_tau=tau_plot)
    g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                         columns=['tau','gamma'])
    Z_res.to_csv(Zoutfile,index=False)
    g_res.to_csv(Goutfile,index=False)

    # pickle the drt object for access to all other attributes
    save_pickle(dr_trunc,pkl)
        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_trunc_Macdonald_0.25.csv
-------------------------------
File fit time: 9.71 seconds
Dumped pickle to map_results\obj_trunc_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_trunc_Macdonald_1.0.csv
-------------------------------
File fit time: 4.50 seconds
Dumped pickle to map_results\obj_trunc_Macdonald_1.0.pkl
-------------------------------
../data/simulated\Z_trunc_Macdonald_2.5.csv
-------------------------------
File fit time: 3.00 seconds
Dumped pickle to map_results\obj_trunc_Macdonald_2.5.pkl
-------------------------------
../data/simulated\Z_trunc_noiseless.csv
-------------------------------
File fit time: 10.19 seconds
Dumped pickle to map_results\obj_trunc_noiseless.pkl
-------------------------------
../data/simulated\Z_trunc_noiseless_FullRange.csv
-------------------------------
File fit time: 25.83 seconds
Dumped pickle to map_results\obj_trunc_noiseless_FullRange.pkl
---------------------------

In [270]:
"MCMC sampling"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in trunc_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    Zoutfile = os.path.join('bayes_results',f'Zout{suffix}')
    Goutfile = os.path.join('bayes_results',f'Gout{suffix}')
    pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix[:-4]))
    if file.split('_')[2]=='noiseless.csv':
        sigma_min=0.005
    else:
        sigma_min=0.002
        
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    df = pd.read_csv(file)
    Z = df['Zreal'].values + 1j*df['Zimag'].values

    # fit
    file_start = time.time()
    dr_trunc.bayes_fit(df['Freq'].values,Z,sigma_min=sigma_min,nonneg_drt=False)
    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

    Z_pred = dr_trunc.predict_Z(df['Freq'].values)
    Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
    sigma_re,sigma_im = dr_trunc.predict_sigma(df['Freq'].values)
    Z_res['sigma_re'] = sigma_re
    Z_res['sigma_im'] = sigma_im

    g_pred = dr_trunc.predict_distribution('DRT',eval_tau=tau_plot)
    # drt credibility interval
    g_lo = dr_trunc.predict_distribution('DRT',eval_tau=tau_plot,percentile=2.5)
    g_hi = dr_trunc.predict_distribution('DRT',eval_tau=tau_plot,percentile=97.5)
    g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                         columns=['tau','gamma','gamma_lo','gamma_hi'])

    Z_res.to_csv(Zoutfile,index=False)
    g_res.to_csv(Goutfile,index=False)

    # pickle the drt object for access to all other attributes
    save_pickle(dr_trunc,pkl)
        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_trunc_Macdonald_0.25.csv
-------------------------------


File fit time: 123.80 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_trunc_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_trunc_Macdonald_1.0.csv
-------------------------------


File fit time: 109.81 seconds


...

File fit time: 136.43 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_trunc_uniform_1.0.pkl
-------------------------------
../data/simulated\Z_trunc_uniform_2.5.csv
-------------------------------


File fit time: 123.67 seconds
Dumped pickle to bayes_results\obj_trunc_uniform_2.5.pkl
Total fit time: 19.93 minutes


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


## DDT

In [54]:
tp_files = glob.glob('../data/simulated/Z_*TP-DDT*.csv')
bp_files = glob.glob('../data/simulated/Z_*BP-DDT*.csv')

tp_ddt = drt.DRT(distributions={'DDT':{'kernel':'DDT','symmetry':'planar','bc':'transmissive','dist_type':'parallel',
                                      'basis_freq':np.logspace(6,-3,91)}
                               }
                )
bp_ddt = drt.DRT(distributions={'DDT':{'kernel':'DDT','symmetry':'planar','bc':'blocking','dist_type':'parallel',
                                      'basis_freq':np.logspace(6,-3,91)} # slightly expand basis from data range to capture full distribution
                               }
                )

In [265]:
"MAP fits"

# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for files,dr in zip([tp_files,bp_files],[tp_ddt,bp_ddt]):

    for file in files:
        print('-------------------------------')
        print(file)
        print('-------------------------------')
        suffix = file[file.find('_'):]
        Zoutfile = os.path.join('map_results',f'Zout{suffix}')
        Goutfile = os.path.join('map_results',f'Gout{suffix}')
        pkl = os.path.join('map_results','obj{}.pkl'.format(suffix[:-4]))
        if file.split('_')[2]=='noiseless.csv':
            sigma_min=0.005
        else:
            sigma_min=0.002
#         if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#             print('Already ran')
#         else:
        df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        # fit
        file_start = time.time()
        dr.map_fit(df['Freq'].values,Z,init_from_ridge=False,sigma_min=sigma_min)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

        # save results
        Z_pred = dr.predict_Z(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
        sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
        Z_res['sigma_re'] = sigma_re
        Z_res['sigma_im'] = sigma_im

        g_pred = dr.predict_distribution('DDT',eval_tau=tau_plot)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)
        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_BimodalTP-DDT_Macdonald_0.05.csv
-------------------------------
File fit time: 5.26 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to map_results\obj_BimodalTP-DDT_Macdonald_0.05.pkl
-------------------------------
../data/simulated\Z_BimodalTP-DDT_Macdonald_0.25.csv
-------------------------------
File fit time: 2.27 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to map_results\obj_BimodalTP-DDT_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_BimodalTP-DDT_noiseless.csv
-------------------------------
File fit time: 3.44 seconds


...

../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to map_results\obj_BimodalBP-DDT_Orazem_0.25.pkl
-------------------------------
../data/simulated\Z_BimodalBP-DDT_uniform_0.05.csv
-------------------------------
File fit time: 3.28 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to map_results\obj_BimodalBP-DDT_uniform_0.05.pkl
-------------------------------
../data/simulated\Z_BimodalBP-DDT_uniform_0.25.csv
-------------------------------
File fit time: 3.51 seconds
Dumped pickle to map_results\obj_BimodalBP-DDT_uniform_0.25.pkl
Total fit time: 0.92 minutes


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [247]:
"MCMC sampling"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for files,dr in zip([tp_files,bp_files],[tp_ddt,bp_ddt]):

    for file in files:
        print('-------------------------------')
        print(file)
        print('-------------------------------')
        suffix = file[file.find('_'):]
        Zoutfile = os.path.join('bayes_results',f'Zout{suffix}')
        Goutfile = os.path.join('bayes_results',f'Gout{suffix}')
        pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix[:-4]))
        if file.split('_')[2]=='noiseless.csv':
            sigma_min=0.005
        else:
            sigma_min=0.002

#         if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#             print('Already ran')
#         else:
        df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values

        # fit
        file_start = time.time()
        dr.bayes_fit(df['Freq'].values,Z,init_from_ridge=False,sigma_min=sigma_min)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

        Z_pred = dr.predict_Z(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,
                             columns=['freq','Zreal','Zimag'])
        sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
        Z_res['sigma_re'] = sigma_re
        Z_res['sigma_im'] = sigma_im

        g_pred = dr.predict_distribution('DDT',eval_tau=tau_plot)
        # drt credibility interval
        g_lo = dr.predict_distribution('DDT',eval_tau=tau_plot,percentile=2.5)
        g_hi = dr.predict_distribution('DDT',eval_tau=tau_plot,percentile=97.5)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                             columns=['tau','gamma','gamma_lo','gamma_hi'])

        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)

print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_BimodalTP-DDT_Macdonald_0.05.csv
-------------------------------


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 175.77 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_BimodalTP-DDT_Macdonald_0.05.pkl
-------------------------------
../data/simulated\Z_BimodalTP-DDT_Macdonald_0.25.csv
-------------------------------


...

../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_BimodalBP-DDT_uniform_0.05.pkl
-------------------------------
../data/simulated\Z_BimodalBP-DDT_uniform_0.25.csv
-------------------------------


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 142.18 seconds
Dumped pickle to bayes_results\obj_BimodalBP-DDT_uniform_0.25.pkl
Total fit time: 33.05 minutes


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


# DRT-DDT

In [326]:
# skip noiseless files
sp_files = glob.glob('../data/simulated/Z_DRT-*-TpDDT_uniform*.csv')
s2p_files = glob.glob('../data/simulated/Z_DRT-TpDDT-BpDDT_uniform*.csv')

# for sp_dr: use xp_scale=0.8
sp_dr = drt.DRT(distributions={'DRT':{'kernel':'DRT','basis_freq':np.logspace(6,-2,81)},
                                'TP-DDT':{'kernel':'DDT','symmetry':'planar','bc':'transmissive','dist_type':'parallel',
                                         'basis_freq':np.logspace(6,-2,81),'x_scale':0.8}
                               }
                )

# for sp2_dr: MAP: xp1_scale = 1 (BP-DDT), xp2_scale = 0.8 (TP-DDT)
# bayes: xp1_scale = 0.5 (BP-DDT), xp2_scale = 0.5 (TP-DDT) (1 and 0.8 also work fine)
s2p_dr = drt.DRT(distributions={'DRT':{'kernel':'DRT'},
            'TP-DDT':{'kernel':'DDT','symmetry':'planar','bc':'transmissive','dist_type':'parallel'},
            'BP-DDT':{'kernel':'DDT','symmetry':'planar','bc':'blocking','dist_type':'parallel'}
                               }
                )

In [328]:
"MAP fits"

# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

# set best basis freq for MAP
s2p_dr.basis_freq = np.logspace(6,-2.4,85)

for files,dr in zip([sp_files],[sp_dr]):
    for file in files:
        print('-------------------------------')
        print(file)
        print('-------------------------------')
        suffix = file[file.find('_'):]
        Zoutfile = os.path.join('map_results',f'Zout{suffix}')
        Goutfile = os.path.join('map_results',f'Gout{suffix}')
        pkl = os.path.join('map_results','obj{}.pkl'.format(suffix[:-4]))
        if file.split('_')[2]=='noiseless.csv':
            sigma_min=0.005
        else:
            sigma_min=0.002
#         if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#             print('Already ran')
#         else:
        df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        # fit
        file_start = time.time()
        dr.map_fit(df['Freq'].values,Z,init_from_ridge=False,sigma_min=sigma_min,nonneg_drt=True,max_iter=1e5)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

        # save results
        Z_pred = dr.predict_Z(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
        sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
        Z_res['sigma_re'] = sigma_re
        Z_res['sigma_im'] = sigma_im

        g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot)
        ftp = dr.predict_distribution('TP-DDT',eval_tau=tau_plot)
        if suffix.find('BpDDT') > 0:
            fbp = dr.predict_distribution('BP-DDT',eval_tau=tau_plot)
            g_res = pd.DataFrame(np.array([tau_plot,g_pred,ftp,fbp]).T,
                                 columns=['tau','gamma','ftp','fbp'])
        else:    
            g_res = pd.DataFrame(np.array([tau_plot,g_pred,ftp]).T,
                                 columns=['tau','gamma','ftp'])

        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)
        
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        Zmod  = np.sqrt(Z*Z.conjugate()).real
        Z_scale = np.std(Zmod)
        Z_star = Z/Z_scale

        Zsmod  = np.sqrt(Z_star*Z_star.conjugate()).real
        Y_star = 1/Z_star
        Ysmod = np.sqrt(Y_star*Y_star.conjugate()).real
        print(np.std(Zsmod), np.std(Ysmod))
        print(dr._Z_scale)
#         print('xs:',np.max(dr._opt_result['xs']))
#         print('xp1_raw:',np.max(dr._opt_result['xp1_raw']))
#         print('xp2_raw:',np.max(dr._opt_result['xp2_raw']))

print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_DRT-2-TpDDT_uniform_0.25.csv
-------------------------------
File fit time: 4.12 seconds
Dumped pickle to map_results\obj_DRT-2-TpDDT_uniform_0.25.pkl
1.0 0.2364923925947181
0.8271966826504259
-------------------------------
../data/simulated\Z_DRT-3-TpDDT_uniform_0.25.csv
-------------------------------
File fit time: 7.53 seconds
Dumped pickle to map_results\obj_DRT-3-TpDDT_uniform_0.25.pkl
1.0 0.22304046487665727
0.8023925697414866
-------------------------------
../data/simulated\Z_DRT-4-TpDDT_uniform_0.25.csv
-------------------------------
File fit time: 5.25 seconds
Dumped pickle to map_results\obj_DRT-4-TpDDT_uniform_0.25.pkl
1.0 0.18768424388506202
0.7431062970991372
Total fit time: 0.31 minutes


In [272]:
"MCMC sampling"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

# set best basis freq for MCMC
s2p_dr.basis_freq = np.logspace(6,-2,81)

for files,dr in zip([s2p_files],[s2p_dr]):
    for file in files:
        print('-------------------------------')
        print(file)
        print('-------------------------------')
        suffix = file[file.find('_'):]
        Zoutfile = os.path.join('bayes_results',f'Zout{suffix}')
        Goutfile = os.path.join('bayes_results',f'Gout{suffix}')
        pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix[:-4]))
        if file.split('_')[2]=='noiseless.csv':
            sigma_min=0.005
        else:
            sigma_min=0.002

#         if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#             print('Already ran')
#         else:
        df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values

        # fit
        file_start = time.time()
        dr.bayes_fit(df['Freq'].values,Z,init_from_ridge=False,sigma_min=sigma_min,nonneg_drt=True)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

        Z_pred = dr.predict_Z(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,
                             columns=['freq','Zreal','Zimag'])
        sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
        Z_res['sigma_re'] = sigma_re
        Z_res['sigma_im'] = sigma_im

        g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot)
        g_lo = dr.predict_distribution('DRT',eval_tau=tau_plot,percentile=2.5)
        g_hi = dr.predict_distribution('DRT',eval_tau=tau_plot,percentile=97.5)

        ftp = dr.predict_distribution('TP-DDT',eval_tau=tau_plot)
        ftp_lo = dr.predict_distribution('TP-DDT',eval_tau=tau_plot,percentile=2.5)
        ftp_hi = dr.predict_distribution('TP-DDT',eval_tau=tau_plot,percentile=97.5)
        if suffix.find('BpDDT') > 0:
            fbp = dr.predict_distribution('BP-DDT',eval_tau=tau_plot)
            fbp_lo = dr.predict_distribution('BP-DDT',eval_tau=tau_plot,percentile=2.5)
            fbp_hi = dr.predict_distribution('BP-DDT',eval_tau=tau_plot,percentile=97.5)
            g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi,
                                           ftp,ftp_lo,ftp_hi,
                                           fbp,fbp_lo,fbp_hi]).T,
                                 columns=['tau','gamma','gamma_lo','gamma_hi',
                                          'ftp','ftp_lo','ftp_hi',
                                          'fbp','fbp_lo','fbp_hi'])
        else:    
            g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi,
                                           ftp,ftp_lo,ftp_hi]).T,
                                 columns=['tau','gamma','gamma_lo','gamma_hi',
                                          'ftp','ftp_lo','ftp_hi'])

        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)

print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_DRT-TpDDT-BpDDT_uniform_0.25.csv
-------------------------------


../bayes-drt\drt.py:2062: RuntimeWarning: overflow encountered in tanh
  # then choose whether to integrate Z_D or Y_D
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 266.32 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_DRT-TpDDT-BpDDT_uniform_0.25.pkl
Total fit time: 4.46 minutes


# Experimental data
## LIB data from DRTtools

In [233]:
lib_files = ['../data/experimental/DRTtools_LIB_data.txt',
             '../data/experimental/DRTtools_LIB_data_qtr.csv'
            ]

lib_basis_freq = np.logspace(4,-5,91)
lib_tau = 1/(2*np.pi*lib_basis_freq)

dr_lib = drt.DRT(basis_freq=lib_basis_freq)

# # run ridge_fit to generate A matrices
# lib_df = pd.read_csv(lib_files[0],sep='\t',header=None)
# lib_df = pd.DataFrame(lib_df.values,columns=['Freq','Zreal','Zimag'])
# Z_lib = lib_df['Zreal'] + 1j*lib_df['Zimag']
# f = lib_df['Freq'].values
# Z_lib = Z_lib
# dr_lib.ridge_fit(f,Z_lib.values)



In [197]:
"MAP fits"
start = time.time()

# finer tau for plotting
tau_plot = np.logspace(np.log10(np.min(lib_tau)),np.log10(np.max(lib_tau)),200)
dr = dr_lib
for file in lib_files:
    print('-------------------------------')
    print('{}: {} distribution(s)'.format(os.path.split(file)[1],len(dr.distributions)))
    print('-------------------------------')
    suffix = file[file.find('_'):-4]

    Zoutfile = os.path.join('map_results',f'Zout{suffix}.csv')
    Goutfile = os.path.join('map_results',f'Gout{suffix}.csv')
    pkl = os.path.join('map_results','obj{}.pkl'.format(suffix))
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    if file[-3:]=='txt':
        df = pd.read_csv(file,sep='\t',header=None)
        df = pd.DataFrame(df.values,columns=['Freq','Zreal','Zimag'])
    else:
        df = pd.read_csv(file)

    Z = df['Zreal'].values + 1j*df['Zimag'].values
    # fit
    file_start = time.time()
    # initializing from the ridge solution may be helpful for faster convergence for more complex spectra
    dr.map_fit(df['Freq'].values,Z,init_from_ridge=False,max_iter=1e5,nonneg_drt=False)
    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

    # save results
    Z_pred = dr.predict_Z(df['Freq'].values)
    Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
    sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
    Z_res['sigma_re'] = sigma_re
    Z_res['sigma_im'] = sigma_im

    g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot)
    g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                         columns=['tau','gamma'])

    Z_res.to_csv(Zoutfile,index=False)
    g_res.to_csv(Goutfile,index=False)

    # pickle the drt object for access to all other attributes
    save_pickle(dr,pkl)

print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
DRTtools_LIB_data.txt: 2 distribution(s)
-------------------------------
File fit time: 7.20 seconds
Dumped pickle to map_results\obj_LIB_data_DRT-TpDDT.pkl
Total fit time: 0.13 minutes


In [234]:
"MCMC sampling"
# finer tau for plotting
tau_plot = np.logspace(np.log10(np.min(lib_tau)),np.log10(np.max(lib_tau)),200)

start = time.time()

for file in lib_files[0:1]:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    # strip file extension
    suffix = suffix[:suffix.find('.')]
    Zoutfile = os.path.join('bayes_results',f'Zout{suffix}.csv')
    Goutfile = os.path.join('bayes_results',f'Gout{suffix}.csv')
    pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix))
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    if file[-3:]=='txt':
        df = pd.read_csv(file,sep='\t',header=None)
        df = pd.DataFrame(df.values,columns=['Freq','Zreal','Zimag'])
    else:
        df = pd.read_csv(file)
    Z = df['Zreal'].values + 1j*df['Zimag'].values

    # fit
    file_start = time.time()
    # initializing from the ridge solution may be helpful for sampling for more complex spectra
    dr_lib.bayes_fit(df['Freq'].values,Z,init_from_ridge=False)
    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

    Z_pred = dr_lib.predict_Z(df['Freq'].values)
    Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,
                         columns=['freq','Zreal','Zimag'])
    sigma_re,sigma_im = dr_lib.predict_sigma(df['Freq'].values)
    Z_res['sigma_re'] = sigma_re
    Z_res['sigma_im'] = sigma_im

    g_pred = dr_lib.predict_distribution('DRT',eval_tau=tau_plot)
    # drt crediibility interval
    g_lo = dr_lib.predict_distribution('DRT',eval_tau=tau_plot,percentile=2.5)
    g_hi = dr_lib.predict_distribution('DRT',eval_tau=tau_plot,percentile=97.5)
    g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                         columns=['tau','gamma','gamma_lo','gamma_hi'])

    Z_res.to_csv(Zoutfile,index=False)
    g_res.to_csv(Goutfile,index=False)

    # pickle the drt object for access to all other attributes
    save_pickle(dr_lib,pkl)
        
print('--------------------------------------------------------')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/experimental/DRTtools_LIB_data.txt
-------------------------------


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 198.61 seconds
Dumped pickle to bayes_results\obj_LIB_data.pkl
--------------------------------------------------------
Total fit time: 3.32 minutes


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


## Protonic ceramic microelectrode

In [2]:
tco_file = '../data/experimental/PDAC_COM3_02109_Contact10_2065C_500C.txt'
tco_df = gt.read_eis_zdata(tco_file)

dr_tco = drt.DRT(basis_freq=np.logspace(7,-3,101))

dr_tco_sp = drt.DRT(distributions={'DRT':{'kernel':'DRT'},
                                  'TP-DDT':{'kernel':'DDT','symmetry':'planar','bc':'transmissive',
                                            'x_scale':2}},
                    basis_freq=np.logspace(7,-3,101))


# finer tau for plotting
tau_plot_tco = np.logspace(-7,4,300)

In [115]:
"MAP fits"
start = time.time()
file = tco_file

dr_tco_sp = drt.DRT(distributions={'DRT':{'kernel':'DRT'},#,'basis_freq':np.logspace(7,1,61)},
                                  'TP-DDT':{'kernel':'DDT','symmetry':'planar','bc':'transmissive',
                                          'x_scale':3}},
                    basis_freq=np.logspace(7,-3,101))

dr_tco_sp.distributions['TP-DDT']['x_scale'] = 3
# dr_tco_sp._recalc_mat=True

for outliers in [True]:#,False]:
    print('-------------------------------')
    print(f'Outliers={outliers}')
    print('-------------------------------')
    for dr in [dr_tco_sp]:
        num_dist = len(dr.distributions)
        print(f'{num_dist} distribution(s)')
        print('-------------------------------')
        suffix = '_PDAC'
        
        if num_dist==2:
            suffix += '_DRT-TpDDT'
        
        if outliers:
            suffix += '_outliers'
        
        Zoutfile = os.path.join('map_results',f'Zout{suffix}.csv')
        Goutfile = os.path.join('map_results',f'Gout{suffix}.csv')
        pkl = os.path.join('map_results','obj{}.pkl'.format(suffix))
    #     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
    #         print('Already ran')
    #     else:
        df = gt.read_eis_zdata(file)
        
        
        
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        # fit
        file_start = time.time()

        if num_dist==2:
            nonneg=True
        else:
            nonneg=False
            
        dr.map_fit(df['Freq'].values,Z,max_iter=1e5,outliers=outliers,nonneg_drt=nonneg)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        # save results
        Z_pred = dr.predict_Z(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
        sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
        Z_res['sigma_re'] = sigma_re
        Z_res['sigma_im'] = sigma_im

        g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot_tco)
        g_res = pd.DataFrame(np.array([tau_plot_tco,g_pred]).T,
                             columns=['tau','gamma'])
        if num_dist==2:
            ftp = dr.predict_distribution('TP-DDT',eval_tau=tau_plot_tco)
            g_res['ftp'] = ftp
            
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

#         pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)

    print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
Outliers=True
-------------------------------
2 distribution(s)
-------------------------------


../bayes-drt\drt.py:2076: RuntimeWarning: overflow encountered in tanh
  return np.tanh(x)/x


File fit time: 13.18 seconds
Dumped pickle to map_results\obj_PDAC_DRT-TpDDT_outliers.pkl
Total fit time: 0.22 minutes


In [116]:
"MCMC sampling"
# tau for plotting
start = time.time()

dr_tco_sp.distributions['TP-DDT']['x_scale'] = 2

file = tco_file
for outliers in [True]:#,False]:
    print('===============================')
    print(f'Outliers={outliers}')
    print('===============================')
    for dr in [dr_tco_sp]:
        num_dist = len(dr.distributions)
        print('-------------------------------')
        print(f'{num_dist} distribution(s)')
        print('-------------------------------')
        suffix = '_PDAC'
        
        if num_dist==2:
            suffix += '_DRT-TpDDT'
        
        if outliers:
            suffix += '_outliers'
        
        Zoutfile = os.path.join('bayes_results',f'Zout{suffix}.csv')
        Goutfile = os.path.join('bayes_results',f'Gout{suffix}.csv')
        pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix))

#         if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#             print('Already ran')
#         else:
        df = gt.read_eis_zdata(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values

        if num_dist==2:
            nonneg=True
        else:
            nonneg=False

        # fit
        file_start = time.time()
        # initializing from the ridge solution may be helpful for sampling for more complex spectra
        dr.bayes_fit(df['Freq'].values,Z,init_from_ridge=False,outliers=outliers,nonneg_drt=nonneg)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

        Z_pred = dr.predict_Z(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,
                             columns=['freq','Zreal','Zimag'])
        sigma_re,sigma_im = dr.predict_sigma(df['Freq'].values)
        Z_res['sigma_re'] = sigma_re
        Z_res['sigma_im'] = sigma_im

        g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot_tco)
        # drt crediibility interval
        g_lo = dr.predict_distribution('DRT',eval_tau=tau_plot_tco,percentile=2.5)
        g_hi = dr.predict_distribution('DRT',eval_tau=tau_plot_tco,percentile=97.5)
        g_res = pd.DataFrame(np.array([tau_plot_tco,g_pred,g_lo,g_hi]).T,
                             columns=['tau','gamma','gamma_lo','gamma_hi'])
        # TP-DDT
        if num_dist==2:
            ftp = dr.predict_distribution('TP-DDT',eval_tau=tau_plot_tco)
            ftp_lo = dr.predict_distribution('TP-DDT',eval_tau=tau_plot_tco,percentile=2.5)
            ftp_hi = dr.predict_distribution('TP-DDT',eval_tau=tau_plot_tco,percentile=97.5)
            g_res['ftp'] = ftp
            g_res['ftp_lo'] = ftp_lo
            g_res['ftp_hi'] = ftp_hi

        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)

print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

Outliers=True
-------------------------------
2 distribution(s)
-------------------------------


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 362.25 seconds
Dumped pickle to bayes_results\obj_PDAC_DRT-TpDDT_outliers.pkl
Total fit time: 6.05 minutes


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
